In [1]:
# Import libraby
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import os

# Folder path
folder_path = "D:/HOC KI 8/3. Graduate project/hasaki_crawling/data"

# Add user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"

# Setting Options
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")
options.add_argument("--no-sandbox")

# Get review counts:
def get_reviews_counts(string):
    start_index = string.find('(')
    end_index = string.find(')')
    review_count = string[start_index + 1:end_index]
    return review_count

In [2]:
# Initialize browsser
driver = webdriver.Chrome(options= options)

# URL to crawl
start_url = "https://hasaki.vn/danh-muc/cham-soc-da-mat-c4.html"

# GET INFOMATION OF ALL ITEMS
# (1, 20+1) ~~ (1-20)
# (21, 40+1) ~~ (21-40)
# (1, 30+1) ~~ (1-30)

df_list = []

for page in range(1, 3):
    
    product_ids, data_product_ids, links, title = [], [], [], []
    price, discount, review_count, quantity_sold, product_variant = [], [], [], [], []

    # Get url
    url = f"{start_url}?p={page}"
    driver.get(url)
    sleep(random.randint(7,8))

    # Get product-id, data-product-id, link
    elems_link_id = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .block_info_item_sp')
    product_ids = [elem.get_attribute('data-id') for elem in elems_link_id] + product_ids
    data_product_ids = [elem.get_attribute('data-product') for elem in elems_link_id] + data_product_ids
    links = [elem.get_attribute('href') for elem in elems_link_id] + links
    sleep(random.randint(1,2))

    # Get title
    elems_title = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .vn_names')
    title = [elem.text for elem in elems_title] + title
    sleep(random.randint(1,2))

    # Get price
    elems_price = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_giacu')
    price = [elem.text for elem in elems_price] + price  
    sleep(random.randint(1,2))

    # Get discount
    elems_discount = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_giamoi')
    discount = [elem.text for elem in elems_discount] + discount
    sleep(random.randint(1,2))

    # Get review counts:
    elems_review_count = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .block_count_by')
    review_count = [get_reviews_counts(elem.text) for elem  in elems_review_count] + review_count
    sleep(random.randint(1,2))

    # Get quantity_sold
    elems_quantity_sold = driver.find_elements(By.CSS_SELECTOR, '.ProductGrid__grid .item_count_by')
    quantity_sold = [elem.text.strip('\n ') for elem  in elems_quantity_sold] + quantity_sold
    sleep(random.randint(1,2))

    # Get product_variant
    elems_product_variant = driver.find_elements(By.CSS_SELECTOR, ".ProductGrid__grid .block_info_item_sp")
    product_variant = [elem.get_attribute('data-variant') for elem  in elems_product_variant] + product_variant
    sleep(random.randint(1,2))

    print(f"Scraped IDs from page {page}")
    sleep(random.randint(7,8))

    # Create dataframe for data crawled
    product_data = pd.DataFrame(
        list(zip(product_ids, data_product_ids, links, title,
            price, discount, quantity_sold, product_variant, review_count
        )),
        columns=[
            'product_id', 'data_product_id', 'link_item', 'title', 
            'original_price', 'current_price', 'quantity_sold', 'product_variant','review_count'
        ]
    )

    df_list.append(product_data)

['19325', '65994', '12684', '86167', '34119', '119084', '99221', '9740', '102959', '11089', '117680', '96523', '43795', '19286', '106681', '106533', '2364', '103543', '89563', '90023', '87859', '105523', '90339', '12064', '102911', '91747', '81465', '90639', '68810', '90401', '66026', '102957', '106679', '43789', '102593', '102185', '3495', '71263', '4359', '76584']
Scraped IDs from page 1
['96589', '93797', '102963', '86161', '100457', '74010', '75956', '7947', '99715', '99707', '94529', '114501', '845', '96659', '117684', '56056', '84667', '98921', '1746', '1278', '95523', '83415', '66054', '95711', '102853', '92917', '96677', '100971', '6258', '79881', '92215', '1742', '102343', '114435', '98637', '101791', '61216', '104259', '89019', '100459']
Scraped IDs from page 2


In [3]:
# Combine all comment crawled
combined_product_data = pd.concat(df_list, ignore_index=True)

In [4]:
# Save into csv
current_datetime = datetime.now().strftime("%Y%m%d_%H%M")
productdata_filename = f"productdata_{current_datetime}.csv"

combined_product_data.to_csv(os.path.join(folder_path, "product", productdata_filename), encoding= "utf-8-sig")
print("All product IDs saved to CSV file.")

All product IDs saved to CSV file.
